In [2]:
import json
import os
from pprint import pprint
import csv
import pandas as pd
import re

Helper Functions

In [3]:
def safe_get(data, path, default=None):
    for key in path:
        if isinstance(data, dict):
            data = data.get(key, default)
        elif isinstance(data, list) and isinstance(key, int) and 0 <= key < len(data):
            data = data[key]
        else:
            return default
    return data

Extracting Authors' info

In [ ]:
#Extract author info
data_dir = 'Project_data'
output_file = 'authors.csv'

# Use a set to track unique author IDs
unique_authors = set()

with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write the header once
    writer.writerow(['at_id', 'name', 'degree'])

    # Walk through all subdirectories and files
    for root, _, files in os.walk(data_dir):
        for filename in files:
            if filename.startswith('20'):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r') as file:
                    print(file_path)
                    try:
                        data = json.load(file)
                        # Access authors
                        authors = data.get('abstracts-retrieval-response', {}).get('authors', {}).get('author', [])
                        for author in authors:
                            at_id = author.get("@auid")
                            if at_id and at_id not in unique_authors:  # Avoid duplicates
                                name = f"{author.get('ce:given-name', '')} {author.get('ce:surname', '')}".strip()
                                degree = author.get('ce:degrees', 'NA')
                                writer.writerow([at_id, name, degree])
                                unique_authors.add(at_id)  # Add the ID to the set
                    except (json.JSONDecodeError, KeyError) as e:
                        print(f"Error processing file {file_path}: {e}")


Extracting Affiliations Info

In [ ]:
#Extract affiliation info
data_dir = 'Sample_data'
output_file = 'affiliations.csv'
unique_affiliations = set()

with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['af_id', 'name', 'organization', 'country', 'city'])
    # Walk through all subdirectories and files
    for root, _, files in os.walk(data_dir):
        for filename in files:
            if filename.startswith('20'):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r') as file:
                    
                    try:
                        data = json.load(file)
                        # Access authors
                        auth_groups = data.get('abstracts-retrieval-response', {}).get('item', {}).get('bibrecord', {}).get('head', {}).get('author-group', [])

                        if isinstance(auth_groups, list):
                            for auth_group in auth_groups:
                                affiliation = auth_group.get('affiliation', {})   
                                af_id = affiliation.get("@afid")
                                if af_id and af_id not in unique_affiliations:  # Avoid duplicates
                                    org_list = affiliation.get('organization', 'NA')
                                    if isinstance(org_list, list):
                                        organization = ', '.join([org['$'] for org in org_list])
                                        name = org_list[-1]['$']
                                    elif isinstance(org_list, dict):
                                        organization = org_list.get('$', 'NA')
                                        name = organization
                                    else:
                                        organization = affiliation.get('ce:text', 'NA')
                                        name = organization
                                    country = affiliation.get('country', 'NA')
                                    city = affiliation.get('city', 'NA')
                                    writer.writerow([af_id, name, organization, country, city])
                                    unique_affiliations.add(af_id)  # Add the ID to the set
                        else:
                            affiliation = auth_groups.get('affiliation', {}) 
                            af_id = affiliation.get("@afid")
                            if af_id and af_id not in unique_affiliations:  # Avoid duplicates
                                org_list = affiliation.get('organization', 'NA')
                                if isinstance(org_list, list):
                                    organization = ', '.join([org['$'] for org in org_list])
                                    name = org_list[-1]['$']
                                elif isinstance(org_list, dict):
                                    organization = org_list.get('$', 'NA')
                                    name = organization
                                else:
                                    organization = affiliation.get('ce:text', 'NA')
                                    name = organization
                            country = affiliation.get('country', 'NA')
                            city = affiliation.get('city', 'NA')
                            writer.writerow([af_id, name, organization, country, city])
                            unique_affiliations.add(af_id)  # Add the ID to the set
                    except (json.JSONDecodeError, KeyError) as e:
                        print(f"Error processing file {file_path}: {e}")

Author to Affiliations

In [6]:
data_dir = 'Project_data'
output_file = 'author_to_affi.csv'
columns = ['pid', 'at_id', 'af_id']
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()
    # Walk through all subdirectories and files
    for root, _, files in os.walk(data_dir):
        for filename in files:
            if filename.startswith('20'):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r') as file:
                    
                    try:
                        data = json.load(file)
                        auth_groups = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'author-group'], None)
                        if isinstance(auth_groups, list):
                            for auth_group in auth_groups:
                                affiliation = auth_group.get('affiliation', {})   
                                af_id = affiliation.get("@afid")
                                authors = auth_group.get('author', [])
                                for author in authors:
                                    at_id = author.get('@auid')
                                    writer.writerow({'pid': filename, 'at_id': at_id, 'af_id': af_id})
                        else:
                            affiliation = auth_groups.get('affiliation', {})
                            af_id = affiliation.get("@afid")
                            authors = auth_groups.get('author', [])
                            for author in authors:
                                at_id = author.get('@auid')
                                writer.writerow({'pid': filename, 'at_id': at_id, 'af_id': af_id})
                    except Exception as e:
                            print(f"Error processing file {file_path}: {e}")

Extract subject area data

In [26]:
data_dir = 'Project_data'
output_file = 'subject_areas.csv'

columns = [
    'subject_area_id', 'subject_area_name',
]

unique_subject = set()


with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()
    # Walk through all subdirectories and files
    for root, _, files in os.walk(data_dir):
        for filename in files:
            if filename.startswith('20'):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    try:
                        data = json.load(file)
                        subject_areas_list = safe_get(data, ['abstracts-retrieval-response', 'subject-areas', 'subject-area'], None)
                        for subject in subject_areas_list:
                            subject_id = subject.get('@code', None)
                            subject_name = subject.get('$', None)
                            if subject_id not in unique_subject:
                                writer.writerow({
                                    'subject_area_id': subject_id,
                                    'subject_area_name': subject_name
                                })
                                unique_subject.add(subject_id)
                    except Exception as e:
                            print(f"Error processing file {file_path}: {e}")



Extracting Paper Data

In [4]:
def get_classification_codes(classification_list):
    """
    Processes the classification list and extracts the relevant classification code.
    
    :param classification_list: List of classifications, can be of different types.
    :return: List of classification codes.
    """
    classification_codes = []
    
    for classification in classification_list:
        if isinstance(classification, dict):
            classification_type = classification.get('@type')
            classification_data = classification.get('classification')
            
            if classification_type == 'SUBJABBR' or classification_type == 'ASJC':
                # For SUBJABBR and AJSC, classification might be a single string or a list
                if isinstance(classification_data, list):
                    classification_codes.extend([item.get('$') for item in classification_data if isinstance(item, dict)])
                else:
                    classification_codes.append(classification_data)  # Single string classification
            
            elif classification_type in ['CPXCLASS', 'FLXCLASS']:
                if isinstance(classification_data, list):
                    classification_codes.extend([item.get('classification-code') for item in classification_data if isinstance(item, dict)])
                elif isinstance(classification_data, dict):
                    classification_codes.append(classification_data.get('classification-code'))
    
    return classification_codes

In [5]:
def check_slice_or_single(data):
    #Ensures that the returned data is always a list, even if it's a single item.
    if data is None:
        return None
    if isinstance(data, list):
        return data
    return [data]

In [6]:
def handleidx(idxterms_data):
    if idxterms_data:
        if isinstance(idxterms_data, dict):  # If idxterms is a dictionary
            mainterm = idxterms_data.get('mainterm')
            if isinstance(mainterm, list):
                idxterms = [i.get('$', None) for i in mainterm]  # Extract the '$' value from each item in the list
            elif isinstance(mainterm, dict):
                idxterms = [mainterm.get('$', None)]  # If 'mainterm' is a single dict, extract the '$'
            else:
                idxterms = None
        elif isinstance(idxterms_data, list):  # If idxterms is a list
            idxterms = [i.get('$', None) for i in idxterms_data if isinstance(i, dict)]  # Loop through and extract '$'
        else:
            idxterms = None
    else:
        idxterms = None
    return idxterms

In [44]:
data_dir = './Sample_data'
output_file = 'papers.csv'

columns = [
    'pid','title', 'pub_date', 'abstract', 'language', 'ref_count',
    'citedby_count', 'corresponding_author', 'author_id', 'subject_areas_id', 'keywords',
    'idxterms', 'classification_code'
]
cnt = 0
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()
    # Walk through all subdirectories and files
    for root, _, files in os.walk(data_dir):
        for filename in files:
            if cnt == 40: break
            cnt+=1
            if filename.startswith('20'):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    try:
                        data = json.load(file)
                        title = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'citation-title'], None)
                        pub_year = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'source', 'publicationdate', 'year'], None)
                        pub_month = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'source', 'publicationdate', 'month'], None)
                        pub_day = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'source', 'publicationdate', 'day'], None)
                        pub_date = f"{pub_day}/{pub_month}/{pub_year}" if pub_year and pub_month and pub_day else None
                        abstract = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'abstracts'], None)
                        language = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'citation-info', 'citation-language', '@language'], None)
                        ref_count = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'tail', 'bibliography', '@refcount'], None)
                        citedby_count = safe_get(data, ['abstracts-retrieval-response', 'coredata', 'citedby-count'], None)
                        
                        
                        authors_list = safe_get(data, ['abstracts-retrieval-response', 'authors', 'author'], None)
                        author_dict= {f"{author.get('ce:given-name', '')} {author.get('ce:surname', '')}".strip() : author.get('@auid', None) for author in authors_list if isinstance(author, dict)} if authors_list else None
                        author_id = list(author_dict.values())
                        cor_author = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'correspondence', 'person'], None)
                        ca_name = f"{safe_get(cor_author, ['ce:given-name'], '')} {safe_get(cor_author, ['ce:surname'], '')}".strip()
                        
                        coressponding_auid = author_dict.get(ca_name, None)

                        subject_areas_list = safe_get(data, ['abstracts-retrieval-response', 'subject-areas', 'subject-area'], None)
                        subject_areas_id = [subject_area.get('@code', None) for subject_area in subject_areas_list if isinstance(subject_area, dict)] if subject_areas_list else None
                        
                        authkeywords = safe_get(data, ['abstracts-retrieval-response', 'authkeywords'], None)
                        keywords = None
                        if authkeywords and 'author-keyword' in authkeywords:
                            authkeyword = authkeywords.get('author-keyword', None)
                            if(isinstance(authkeyword, dict)):
                                keywords = re.sub(r'(?<=[\s])([A-Z])', r' \1', authkeyword.get('$', None)).split('  ')
                            else:
                                keywords = [k['$'] for k in authkeyword if isinstance(k, dict)]                            
                        keywords = check_slice_or_single(keywords)
                        
                        idxterms_data = safe_get(data, ['abstracts-retrieval-response', 'idxterms'], None)
                        idxterms = handleidx(idxterms_data)
                        
                        classification_list = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'enhancement', 'classificationgroup', 'classifications'], None)
                        classification_code = get_classification_codes(classification_list)
                        classification_code = check_slice_or_single(classification_code)
                        
                        writer.writerow({
                            'pid': filename,
                            'corresponding_author': coressponding_auid,
                            'title': title,
                            'pub_date': pub_date,
                            'abstract': abstract,
                            'language': language,
                            'ref_count': ref_count,
                            'citedby_count': citedby_count,
                            'author_id': author_id,
                            'subject_areas_id': subject_areas_id,
                            'keywords': keywords,
                            'idxterms': idxterms,
                            'classification_code': classification_code
                        })
                    except Exception as e:
                            print(f"Error processing file {file_path}: {e}")



In [ ]:
#--- Phuree---#

from geopy.geocoders import Nominatim
from pprint import pprint
geolocator = Nominatim(user_agent="isdjuiodfgdfjnjf847hn5")

data_dir = './Sample_data'
output_file = '../CSVs/papers-mock.csv'


cnt = 0

for root, _, files in os.walk(data_dir):
    for filename in files:
        if cnt == 50: break
        cnt+=1
        if filename.startswith('20'):
            file_path = os.path.join(root, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                try:
                    data = json.load(file)
                    
                    # Extract organization (institution) information
                    org_list = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'correspondence','affiliation','organization'], None)
                    organization = None
                    if isinstance(org_list, list):
                        organization = org_list[-1]['$'] if org_list else None
                    elif isinstance(org_list, dict):
                        organization = org_list.get('$', None)

                    # city and cocity
                    city_file = '../CSVs/city1.csv'
                    #! Create new city.csv if it doesn't exist
                    if not os.path.exists(city_file):
                        city_df = pd.DataFrame(columns=['city_id', 'city', 'country', 'citation_sum', 'p_count','lat','lon'])
                        city_dict = {}
                        # Add a mock row
                        mock_row = {
                            'city_id': 1,
                            'city': 'Mock City eiei',
                            'country': 'Mock Country eiei',
                            'citation_sum': 0,
                            'p_count': 0,
                            'lat': 0,
                            'lon': 0
                        }
                        city_df.loc[-1] = mock_row
                    else:
                        city_df = pd.read_csv(city_file)

                    city_dict = {}
                    if city_df.shape[0] != 0:
                        city_dict = {row['city']: row['city_id'] for _, row in city_df.iterrows()}

                    citedby_count = safe_get(data, ['abstracts-retrieval-response','coredata','citedby-count'], None)
                    citedby_count = int(citedby_count) if citedby_count else 0
                    # print("count",citedby_count)
                    # set (city, country)
                    city_country_set = set()
                    
                    affiliations = safe_get(data, ['abstracts-retrieval-response', 'affiliation'], None)
                    # always return list or None
                    affiliation_list = check_slice_or_single(affiliations) 
            
                    for affiliation in affiliation_list:
                        city = affiliation.get('affiliation-city', None)
                        country = affiliation.get('affiliation-country', None)
                        # print(city, country)
                        if city and country:
                            city_country_set.add((city, country))

                    # Handle I. Single City, II. Multiple Cities
                    if len(city_country_set) == 1:
                        #! Case I: Single City
                        city, country = list(city_country_set)[0]
                        
                        if city in city_dict:
                            city_id = city_dict[city] 
                            city_df.loc[city_df['city_id'] == city_id, 'citation_sum'] += int(citedby_count)
                            city_df.loc[city_df['city_id'] == city_id, 'p_count'] += 1
                        else:
                            city_id = len(city_dict) + 1
                            city_dict[city] = city_id
                            #geolocation 
                            # geo = geolocator.geocode(f"{city}, {country}")
                            lat, lon = None, None
                            # if geo:
                            #     lat, lon = geo.latitude, geo.longitude
                                
                            new_row = pd.DataFrame([{
                                'city_id': city_id,
                                'city': city,
                                'country': country,
                                'citation_sum': int(citedby_count),
                                'p_count': 1,
                                'lat': lat,
                                'lon': lon
                            }])
                            city_df = pd.concat([city_df, new_row], ignore_index=True)
                    else:
                        #! Case II: Multiple Cities
                        for city, country in city_country_set:
                            if city in city_dict:
                                city_id = city_dict[city]
                                city_df.loc[city_df['city_id'] == city_id, 'citation_sum'] += int(citedby_count)
                                city_df.loc[city_df['city_id'] == city_id, 'p_count'] += 1
                            else:
                                city_id = len(city_dict) + 1
                                city_dict[city] = city_id
                                #geolocation 
                                # geo = geolocator.geocode(f"{city}, {country}")
                                lat, lon = None, None
                                # if geo:
                                #     lat, lon = geo.latitude, geo.longitude
                                new_row = pd.DataFrame([{
                                    'city_id': city_id,
                                    'city': city,
                                    'country': country,
                                    'citation_sum': int(citedby_count),
                                    'p_count': 1,
                                    'lat': lat,
                                    'lon': lon
                                }])
                                city_df = pd.concat([city_df, new_row], ignore_index=True)

                        # create cocity.csv and link the cities
                        cocity_file = '../CSVs/cocity.csv'
                        with open(cocity_file, 'a', newline='', encoding='utf-8') as cocity_csvfile:
                            cocity_writer = csv.writer(cocity_csvfile)
                            # Write header if file is empty
                            if os.stat(cocity_file).st_size == 0:
                                cocity_writer.writerow(['city_id1', 'city_id2', 'filename'])
                            for city1, country1 in city_country_set:
                                for city2, country2 in city_country_set:
                                    if city1 != city2: # Avoid linking same city
                                        city_id1 = city_dict[city1]
                                        city_id2 = city_dict[city2]
                                        cocity_writer.writerow([city_id1, city_id2, filename])


                    city_df.to_csv(city_file, index=False)
                     
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")


In [ ]:
from geopy.geocoders import Nominatim
from pprint import pprint
import requests
import wikipediaapi
import world_bank_data as wb
import pandas as pd
from timezonefinder import TimezoneFinder
import langcodes

geolocator = Nominatim(user_agent="geoapiExeddsdsddsrcises111")

city = "Bangkok"
country = "Thailand"

data = geolocator.geocode(f"{city}, {country}")
pprint(data.raw)

#http://api.geonames.org/searchJSON?name=Providence&maxRows=1&username=phuree


In [ ]:
import requests
from pprint import pprint
import pandas as pd
from timezonefinder import TimezoneFinder
import langcodes
from bs4 import BeautifulSoup
# Replace these with your own API keys
GEONAMES_USERNAME = "phuree"

def get_gdp_per_capita(country_code):
    """Fetch GDP per capita for a country from World Bank."""
    url = f"http://api.worldbank.org/v2/country/{country_code}/indicator/NY.GDP.PCAP.CD?format=json"
    response = requests.get(url)
    if response.ok and len(response.json()) > 1:
        data = response.json()[1]
        return data[0]['value'] if data else None
    return None

def get_time_zone(lat, lng):
    try:
        tf = TimezoneFinder()
        return tf.timezone_at(lng=lng, lat=lat)
    except Exception as e:
        print(f"Error fetching time zone data: {e}")

def get_population(city, country):
    """Fetch latitude and longitude of a city using GeoNames."""
    url = f"http://api.geonames.org/searchJSON?name={city}&maxRows=1&username={GEONAMES_USERNAME}"
    response = requests.get(url)
    if response.ok:
        data = response.json()
        # print(data)
        if data['geonames']:
            return data['geonames'][0]['population']
    return None

def get_safety_index(city):
    """Scrape safety index from Numbeo website."""
    city_query = city.replace(" ", "-")
    url = f"https://www.numbeo.com/crime/in/{city_query}"
    response = requests.get(url)

    if response.ok:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find the safety index value in the page
        table = soup.find("table", {"class": "table_indices"})
        if table:
            safety_index = table.find_all("tr")[2].find_all("td")[1].text.strip()
            return safety_index
    return None

df = pd.read_csv('../CSVs/city.csv')
cnt = 0
for index, row in df.iterrows():
    if cnt == 6: break
    cnt+=1
    city = row['city']
    country = row['country']
    city_id = row['city_id']
    lat = row['lat']
    lng = row['lon']
    population = get_population(city, country)
    gdp_per_capita = get_gdp_per_capita(country)
    time_zone = get_time_zone(lat, lng) if lat and lng else None
    safety_index = get_safety_index(city)

    print(f"City: {city}")
    print(f"Country: {country}")
    print(f"Population: {population}")
    print(f"GDP per Capita: {gdp_per_capita}")
    print(f"Latitude: {lat}")
    print(f"Longitude: {lng}")
    print(f"Time Zone: {time_zone}")
    print(f"Safety Index: {safety_index}")
    print()



Error fetching language data: Can't find any language named 'Mock Country eiei'
City: Mock City eiei
Country: Mock Country eiei
Population: None
GDP per Capita: None
Latitude: 0.0
Longitude: 0.0
Time Zone: None
Primary Language: None
Safety Index: None

Error fetching language data: Can't find any language named 'Viet Nam'
City: Ho Chi Minh City
Country: Viet Nam
Population: 8993082
GDP per Capita: None
Latitude: 10.7763897
Longitude: 106.7011391
Time Zone: Asia/Ho_Chi_Minh
Primary Language: None
Safety Index: 48.52



KeyboardInterrupt: 